In [ ]:
# IMPORTS AND SETTINGS

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import jaccard_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
from tqdm._tqdm_notebook import tqdm_notebook
%config IPCompleter.greedy=True
%config Completer.use_jedi = False
%load_ext line_profiler

In [ ]:
# to make display(...) show all rows of a dataframe
pd.set_option('display.max_rows', None)

In [ ]:
pd.reset_option('display.max_rows')

In [ ]:
# FUNCTION DEFS

# extract string movieId from movieclipId (without leading zeros)
def str_movie_id(movieclip_id):
    return str(movie_id(movieclip_id))

# extract int movieId from movieclipId
def movie_id(movieclip_id):
    return int(movieclip_id.split('_')[0])

# calculate precision and tag coverage for the results of a single query
def rate(qresults, ks):
    qmovie_id = qresults.movieId.head(1)    
    rating = pd.DataFrame(index=[qmovie_id])
    for k in ks:
        rmovie_ids = [movie_id(y) for y in qresults.movieclipId.head(k)]
        # rate relevance    
        qgenres = genres.loc[qmovie_id]
        rgenres = genres.loc[rmovie_ids]
        rjscore = rgenres.apply(lambda row: jaccard_score(qgenres.values[0], row), axis=1)
        prec = len(rjscore[rjscore > 0.5]) / len(rjscore)
        rating['P@' + str(k)] = prec
        # rate diversity
        qtags = tags.loc[qmovie_id].iloc[0]
        qtags = qtags[qtags > 0].index
        rtags = tag_occs.loc[rmovie_ids].agg('sum')
        rtags = rtags[rtags > 0]
        urtags = rtags.index
        rtags = np.repeat(rtags.index, rtags.values)
        abs_tag_cov = len(urtags)
        rel_tag_cov = len(urtags) / len(qtags)
        rating['AbsTagCov@' + str(k)] = abs_tag_cov
        rating['RelTagCov@' + str(k)] = rel_tag_cov
        entropy = entropy_label_distribution(rtags) # TODO rtags should not be unique!
        rating['Entropy@' + str(k)] = entropy
    return rating

# Compute entropy of label distribution
def entropy_label_distribution(labels):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0

    value, counts = np.unique(labels, return_counts=True)
    probs = counts / np.float32(n_labels)
    n_classes = np.count_nonzero(probs)

    if n_classes <= 1:
        return 0.0

    # Compute entropy
    ent = 0.0

    for p in probs:
        ent -= p * np.log(p)

    return ent

In [ ]:
# RESULTS FILE EVALUATION 

results = pd.read_csv('results.csv')    
results_per_query = results.groupby('movieId')
tqdm_notebook.pandas(desc='rating each query result...')
# TODO this is super slow...
ratings = results_per_query.progress_apply(lambda x: rate(x, [5, 10, 30, 50]))
ratings.index = ratings.index.get_level_values(0)
agg_ratings = ratings.agg(['mean', 'median', 'min', 'max'])
display(agg_ratings)

In [ ]:
# DATA PARSING AND PREPARATION

# parse data
movies = pd.read_csv('MMSR_dataset_2019/devset_movies.csv', index_col='movieId')
clips = pd.read_csv('MMSR_dataset_2019/devset_ids.csv', index_col='movieId')
tags = pd.read_csv('MMSR_dataset_2019/features/Metadata/devset_TagFeatures.csv', index_col='movieId')
genres = pd.read_csv('MMSR_dataset_2019/features/Metadata/devset_GenreFeatures.csv', index_col='movieId')
# TODO use all BLFs instead of just SPECTRAL (I don't have enough RAM for that...)
blfs = pd.read_csv('MMSR_dataset_2019/features/Audio/Block level features/Component6/BLF_SPECTRAL_fullId.csv', header=None, index_col=0)
alexnet = pd.read_csv('MMSR_dataset_2019/features/Visual/Deep AlexNetFc7/Avg/AlexNetFeatures - AVG - fc7.csv', header=None, index_col=0)

# fix missing or superfluous movies or clips in features
tags = movies.join(tags).drop('title', axis=1).fillna(0)
genres = movies.join(genres).drop('title', axis=1).fillna(0)
blfs = clips.join(blfs, on='movieclipId').set_index('movieclipId').fillna(0)
alexnet = clips.join(alexnet, on='movieclipId').set_index('movieclipId').fillna(0)

# normalize and reduce dimensionality...
scaler = MinMaxScaler()
pca = PCA(n_components=0.9)
red_tags = pd.DataFrame(pca.fit_transform(scaler.fit_transform(tags)), index=tags.index)
blfs = pd.DataFrame(pca.fit_transform(scaler.fit_transform(blfs)), index=blfs.index)
alexnet = pd.DataFrame(pca.fit_transform(scaler.fit_transform(alexnet)), index=alexnet.index)

# precompute tag occurrences for calculating tag coverage
tag_occs = tags > 0

In [ ]:
# DECENT? SOLUTION USING BLFS AND ALEXNET 

results = pd.DataFrame(columns=['movieId', 'movieclipId', 'sim', 'rank']).set_index(['movieId', 'movieclipId'])

# 1) compute single feature vector for metadata, audio and visual features for (all clips of) a movie
qblfs = blfs.groupby(lambda x: movie_id(x)).mean()
qalexnet = alexnet.groupby(lambda x: movie_id(x)).mean()

for movieId in tqdm_notebook(movies.index):
    # 2) rank clips by computing distance to aggregated feature vectors of this movie
    ranked_clips_audio = pd.DataFrame(euclidean_distances(blfs.values, qblfs.loc[movieId].values.reshape(1, -1)), 
                                        index=clips.movieclipId, 
                                       columns=['dist']).sort_values('dist')
    ranked_clips_visual = pd.DataFrame(euclidean_distances(alexnet.values, qalexnet.loc[movieId].values.reshape(1, -1)),
                                        index=clips.movieclipId, 
                                        columns=['dist']).sort_values('dist')
    
    # 3) take most similar clips (alternate between audio and visual features) but at most 3 of the same movie
    qresult = pd.DataFrame(columns=['movieclipId', 'sim']).set_index('movieclipId')
    movie_counter = pd.DataFrame(np.zeros(len(movies)), index=movies.index, columns=['counter'])
    while len(qresult) < 100:
        source = ranked_clips_audio if len(qresult) % 2 == 0 else ranked_clips_visual
        for i in range(0, len(source)):
            rcandidate = source.iloc[i]
            is_new_clip = not rcandidate.name in qresult.index
            is_movie_allowed = movie_counter.loc[movie_id(rcandidate.name)].counter < 3
            if is_new_clip & movie_allowed:
                if len(qresult) % 2 == 0:
                    ranked_clips_audio = source.iloc[i:]
                else:
                    ranked_clips_visual = source.iloc[i:]
                qresult.loc[rcandidate.name] = rcandidate.dist
                break
    
    # fixup ranks and movieId
    qresult['rank'] = np.arange(100)
    qresult['movieId'] = np.full(100, movieId)
    # transform euclidean distance into similarity measure
    qresult['sim'] = qresult['sim'].map(lambda x: 1 / (1 + x))
    results = pd.concat([results, qresult], sort=False)
    
results.reset_index(inplace=True)
results.columns = ['movieclipId', 'sim', 'rank', 'movieId']
results = results.astype({'movieId':int})
results.to_csv('results.csv', columns=['movieId', 'movieclipId', 'sim', 'rank'], index=False)

In [ ]:
# BAD SOLUTION USING ONLY METADATA

# compute cosine similarities between tag tf_idf vectors
tags_cos_sim = pd.DataFrame(cosine_similarity(tags.values),
                            columns=tags.index.values, 
                            index=tags.index)

# generate results file
results = pd.DataFrame(columns=['movieId', 'movieclipId', 'sim', 'rank'])

for movieId in tqdm_notebook(movies.index):
    sim = tags_cos_sim[[movieId]]
    sim = sim.reset_index()
    sim = sim.rename(columns={movieId : 'sim', 'movieId': 'otherMovieId'})
    sim = sim.join(clips, on='otherMovieId', how='outer')
    sim = sim.reset_index(drop=True)
    sim = sim.drop('otherMovieId', axis=1)
    sim = sim.sort_values('sim', ascending=False)
    sim = sim.head(100)
    sim['rank'] = np.arange(100)
    sim['movieId'] = np.full(100, movieId)
    results = pd.concat([results, sim], ignore_index=True)
results.to_csv('results.csv', columns=['movieId', 'movieclipId', 'sim', 'rank'], index=False)

